# PyRATES Reproducibility Study

## Koslow et al. (2011) "Impact of declining intermediate-water oxygen on deepwater fishes in the California Current"
[Link to study](https://www.researchgate.net/publication/263583233_Impact_of_declining_intermediate-water_oxygen_on_deepwater_fishes_in_the_California_Current)

***

### 0. Set Up

Load libraries.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

***

### 1. Get and Process CalCOFI Ichthyoplankton Data

CalCOFI ichthyoplankton data is hosted here on [ERDDAP](https://coastwatch.pfeg.noaa.gov/erddap/tabledap/erdCalCOFIlrvcnt.html).

The analysis in this paper includes data from 3 species' eggs in addition to the ichthyoplankton data. As of last check on 7/30/24, the link to the CalCOFI egg count positive tows dataset on [calcofi.org](https://calcofi.org/data/marine-ecosystem-data/fish-eggs-larvae/) is giving a 404 error. For now I will just work with the ichthyoplankton data.

Manual step: query ERDDAP.
- check all variables
- constrain the time >= 1951-01-18T00:00:00Z and <= 2008-12-31T23:59:59Z
- download the data as a csv file

Read in the data. !!Adjust this later to read from the github filepath!!

In [ ]:
# there is a datetime column; use this as the index
df = pd.read_csv("erdCalCOFIlrvcnt_492a_ca36_1eb4.csv",
                 index_col = 7, parse_dates = True)

Check the column names of the data.

In [ ]:
display(df.columns)

View the first few rows of the data.

In [ ]:
display(df.head())

Select just the needed columns: line, station, scientific name, and larval abundance.

In [ ]:
# get just the columns we need
df = (df.filter(items =["line","station","scientific_name", "larvae_10m2"])
        .dropna())

# show the first few observations in the new dataframe
display(df.head())

Build a dataframe with the lines and stations used in the study.

Note that the text says that 51 stations were used; however, the map in Figure 1 seems to have 55 stations in the outlined region. I am including all 55 of these stations because it is not clear which ones were removed from the study.

In [ ]:
# create a DataFrame with lines as columns and stations as rows
stations = pd.DataFrame(
   {"76.7" : [80, 70, 60, 55, 51, 49, pd.NA, pd.NA, pd.NA, pd.NA, pd.NA, pd.NA],
    "80" : [100, 90, 80, 70, 60, 55, 51, pd.NA, pd.NA, pd.NA, pd.NA, pd.NA],
    "81.8" : [46.9, pd.NA, pd.NA, pd.NA, pd.NA, pd.NA, pd.NA, pd.NA, pd.NA, pd.NA, pd.NA, pd.NA],
    "83.3" : [90, 80, 70, 60, 55, 51, 42, 40.6, pd.NA, pd.NA, pd.NA, pd.NA],
    "86.7" : [90, 80, 70, 60, 55, 50, 45, 40, 35, 33, pd.NA, pd.NA],
    "90" : [100, 90, 80, 70, 60, 53, 45, 37, 35, 30, 28, pd.NA],
    "93.3" : [90, 80, 70, 60, 55, 50, 45, 40, 35, 30, 28, 26.7]
    }
)

# melt the df to get a row for each line-station combination
stations = (pd.melt(stations,
                   var_name = "line",
                   value_name = "station")
               .dropna() # now drop the NA values
               .astype('float64') # coerce to float64 to facilitate merging later
               )

# count the number of stations to check there are 55
display(len(stations))

Filter for the stations used in the study.

In [ ]:
# inner join to keep only the rows with the correct line-stations we are keeping
df2 = (df.reset_index() # move the datetime index to a regular column
        .merge(stations,
               how = "inner",
               left_on = ["line", "station"],
               right_on = ["line", "station"]) # do the merge
        .set_index("time")) # reset the datetime column to index

# count the number of observations at each line-station
observations = df2.groupby(["line","station"]).size()

# count again to check there are still 55 stations
display(len(observations))

Filter for only taxa identified to species... is what I did originally but now I see the authors have kept some larger taxonomic groupings, so I will skip this step.

In [ ]:
# since species names are "Genus species", they all contain a space, so I will filter the scientific_name column for just the ones that contain a space
# df3 = df2[df2["scientific_name"].str.contains(" ")]

df3 = df2

# count the number of TAXA currently included
display(len(df3["scientific_name"].unique()))

Assign seasons to the dates. In the text, it says Jan-Feb = winter, Mar-May = spring, Jun-Aug = summer, Sep-Dec = fall.

In [ ]:
# make sure the index is of datetime type
df3.index = pd.to_datetime(df3.index, utc = True)

# code the seasons based on the month column
df3.loc[df3.index.month.isin([1,2]),"season"] = "Winter"
df3.loc[df3.index.month.isin([3,4,5]),"season"] = "Spring"
df3.loc[df3.index.month.isin([6,7,8]),"season"] = "Summer"
df3.loc[df3.index.month.isin([9,10,11,12]),"season"] = "Fall"

# show the last few rows
display(df3.tail())



Remove years with < three seasons of data.

In [ ]:
# make a new column with the index year
df3["year"] = df3.index.year

# for each year, count the number of seasons of data present
seasons = (df3.filter(items = ["year", "season"])
           .groupby("year") # group by year
           .nunique() # count the seasons each year has
           .rename(columns = {"season": "count"}) # name the count column "count"
           .reset_index() # ungroup
           )

# drop the years that have less than 3 seasons of data
df3 = df3.drop(
    df3[df3.year.isin(seasons.query("count < 3")["year"])].index # get the index values matching this query
    ) # then drop those rows

# print the years remaining and the years that have been dropped
display(df3["year"].unique())
display(seasons.query("count < 3")["year"])

Calculate seasonal means (for each year, each season) of larval abundance.

In [ ]:
# make sure the abundance column is numeric
df3["larvae_10m2"] = pd.to_numeric(df3["larvae_10m2"])

# print the data types
display(df3.dtypes)

In [ ]:
# group by year, season, and taxa. then take the mean of the larval abundance
seasonalMeans = (df3.filter(["year","season","scientific_name","larvae_10m2"])
                 .groupby(["year","season","scientific_name"]) # group by year, season, taxa
                 .mean() # take the mean of larvae_10m2
                 .reset_index() # ungroup
                 )

# show the first few rows
display(seasonalMeans.head())

Calculate annual means of larval abundance.

In [ ]:
# group by year and taxa, then take the mean of the seasonal abundances
annualMeans = (seasonalMeans.filter(["year", "scientific_name", "larvae_10m2"])
               .groupby(["year", "scientific_name"]) # group by year and taxa
               .mean() # take the mean of larvae_10m2
               .reset_index() # ungroup
               )

# show the last few rows
display(annualMeans.tail())

Remove species with data in < half the years. We should end up with larval abundance observations for roughly 86 taxa.

In [ ]:
# for each taxon, count how many years of data they have
yearsOfData = (annualMeans.filter(["year", "scientific_name"])
               .groupby(["scientific_name"]) # group by taxa
               .nunique() # count the number of years each taxon has
               .rename(columns = {"year": "count"}) # name the count column "count"
               .reset_index() # ungroup
               )

# drop taxa with years of data less than the threshold (29, half the length of the time series)
annualMeans = annualMeans.drop(
    annualMeans[annualMeans.scientific_name.isin(
        yearsOfData.query("count < 29")["scientific_name"]
        )].index # get the index values matching this query
    ) # then drop those rows

# count the taxa remaining
display(len(annualMeans["scientific_name"].unique()))

We still have 93 taxa. Besides occurrence in at least half the years, the other condition stated in the paper was "consistent identification and enumeration since 1951". Let's plot the timeseries of annual abundance for each species.

In [ ]:
# pivot_longer to get a column for each species
annualMeansPlot = annualMeans.pivot_table(values="larvae_10m2", index="year", columns="scientific_name")

# plot set up
fig, axs = plt.subplots(nrows = 10, ncols = 10, # layout of subplots
                        figsize = (30,45)) # figure size
fig.subplots_adjust(wspace = 0.3, hspace = 0.3, bottom = 0.115, top = 0.88) # adjust white space in figure
fig.suptitle("Annual Average Larval Abundances 1951-2008", fontsize = 20, y = 0.9) # set overall title
fig.supxlabel("Year", fontsize = 17, y = 0.1) # set overall x axis label
fig.supylabel("Larval Abundance 10/m^2", fontsize = 17, x = 0.1) # set overall y axis label
plt.setp(axs, xlim = (1950, 2009)) # set overall x axis limits

# populate the subplots
for col, ax in zip(annualMeansPlot.columns, axs.ravel()):
    annualMeansPlot[[col]].plot(ax = ax)

    ax.set_title(col, style = "italic") # subplot title = taxon name in italics
    ax.get_legend().remove() # no subplot legend
    ax.set_xlabel("", fontsize = 16) #blank subplot xlabel

# delete empty subplots
for i in range(3,10):
    fig.delaxes(axs[9, i])

# display the figure
plt.show()

# this cell should take 10-15 secs to run

I will drop 'Teleostei' because that grouping is too broad. Then I will leave the rest of the taxa in.

In [ ]:
# placeholder for dropping additional taxa
annualMeans = annualMeans[annualMeans["scientific_name"] != "Teleostei"]

# count the taxa remaining, there should be 92
display(len(annualMeans["scientific_name"].unique()))

Log-transform the annual means.

In [ ]:
# apply log transformation to larvae_10m2
annualMeans["log10_larvae"] = np.log10(annualMeans["larvae_10m2"])

# show the first few columns
display(annualMeans.head())

***

### 2. Get and Process CalCOFI Oxygen Data

CalCOFI midwater oxygen data is included in the [Bottle Database](https://calcofi.org/data/oceanographic-data/bottle-database/).

Manual step: download CalCOFI bottle database and save as csv file.

Read in the data.

In [ ]:
# the fourth column, Depth_ID, will be used as the index
oxygen = pd.read_csv("~/Desktop/PyRATES/pyrates/calcofi-bottle.csv",
                     index_col = 3)

Check the column names of the data.

In [ ]:
display(oxygen.columns)

View the first few rows of the data.

In [ ]:
display(oxygen.head())

Subset for only the columns we need-- Sta_ID (contains line and station), Depthm, O2ml_L.

In [ ]:
# get just the columns we need and drop NA values
oxygen = (oxygen.filter(items =["Sta_ID","Depthm", "O2ml_L"])
                .dropna())

# show the first few observations in the new dataframe
display(oxygen.head())

Filter for the years 1951-2008. Date identifiers are included in the Depth_ID index; the first five characters are [Century]-[Year].

In [ ]:
# extract the first five characters of the Depth_ID string and put them in a new column Year
oxygen = oxygen.assign(Year = oxygen.index.str[:5]) 

# remove the "-" character from the Year column
oxygen["Year"] = oxygen["Year"].str.replace(r'\-', '', regex=True)

# make the Year column numeric
oxygen["Year"] = pd.to_numeric(oxygen["Year"])

# filter for the years 1951-2008 and save the result in a new df
oxygen2 = oxygen.query("Year >= 1951 & Year <= 2008")

# show the last few rows
display(oxygen2.tail())

Filter for the select stations used in this study.

In [ ]:
# split the "Sta_ID" column into a "line" column and "station" column and make those columns floats
oxygen2[["line", "station"]] = oxygen2["Sta_ID"].str.split(' ', n = 1, expand = True).astype('float64')

# inner join with the stations dataframe to keep only the rows with the correct line-stations
oxygen3 = (oxygen2.reset_index() # move the index to a regular column
                  .merge(stations,
                         how = "inner",
                         left_on = ["line", "station"],
                         right_on = ["line", "station"]) # do the merge
                  .set_index("Depth_ID")) # reset the Depth_ID column to index

# count the number of observations at each line-station
oxygenObservations = oxygen3.groupby(["line","station"]).size()

# count again to check there are 55 stations of data
display(len(oxygenObservations))

# show the first few rows
display(oxygen3.head())

Filter for depths 200-400m.

In [ ]:
# query the Depthm column for values between 200 and 400
oxygen3 = oxygen3.query("Depthm >= 200 & Depthm <= 400")

# show the first few rows
display(oxygen3.head())

Assign seasons to the data.

In [ ]:
# extract the month from the Depth_ID index
oxygen3 = oxygen3.assign(Month = oxygen3.index.str[5:7])
oxygen3["Month"] = pd.to_numeric(oxygen3["Month"]) # make numeric

# code the seasons based on the month column
oxygen3.loc[oxygen3.Month.isin([1,2]),"season"] = "Winter"
oxygen3.loc[oxygen3.Month.isin([3,4,5]),"season"] = "Spring"
oxygen3.loc[oxygen3.Month.isin([6,7,8]),"season"] = "Summer"
oxygen3.loc[oxygen3.Month.isin([9,10,11,12]),"season"] = "Fall"

# show the last few rows
display(oxygen3.tail())

Remove years with < three seasons of data.

In [ ]:
# for each year, count the number of seasons of data present
oxygenSeasons = (oxygen3.filter(items = ["Year", "season"])
           .groupby("Year") # group by year
           .nunique() # count the seasons each year has
           .rename(columns = {"season": "count"}) # name the count column "count"
           .reset_index() # ungroup
           )

# drop the years that have less than 3 seasons of data
oxygen3 = oxygen3.drop(
    oxygen3[oxygen3.Year.isin(oxygenSeasons.query("count < 3")["Year"])].index # get the index values matching this query
    ) # then drop those rows

# print the years remaining and the years that have been dropped
display(oxygen3["Year"].unique())
display(oxygenSeasons.query("count < 3")["Year"])

Calculate seasonal means of midwater oxygen.

In [ ]:
# group by year and season. then take the mean of the oxygen
seasonalOxygen = (oxygen3.filter(["Year","season", "O2ml_L"])
                         .groupby(["Year","season"]) # group by year, season
                         .mean() # take the mean of O2ml_L
                         .reset_index() # ungroup
                 )

# show the first few rows
display(seasonalOxygen.head())

Calculate annual means of midwater oxygen.

In [ ]:
# calculate annual means
annualOxygen = (seasonalOxygen.filter(["O2ml_L", "Year"]) # select oxygen and year columns
                              .groupby("Year") # group by year
                              .mean() # take the mean of O2ml_L
                              .reset_index() # ungroup
                )

# show the first few rows
display(annualOxygen.head())

Plot annual midwater oxygen time series.

In [ ]:
fig = plt.figure(figsize = (10, 6)) # set figure size

ax = fig.add_subplot(1, 1, 1) # subplot layout
ax.plot(annualOxygen["Year"], annualOxygen["O2ml_L"]); # variables to plot

ax.set_xlabel("Year") # set x axis label
ax.set_ylabel("Mean Midwater Oxygen (ml/L)") # set y axis label
ax.set_title("Annual Mean Oxygen, 200-400m") # set figure title

***

### 3. Analysis

Standardize taxa by mean and standard deviation.

In [ ]:
# take the annualMeans df, cast it so that each taxon is a column
pca_data = (annualMeans.filter(["year", "scientific_name", "log10_larvae"]) # select variables
                       .pivot_table(values = "log10_larvae",
                                    index = "year",
                                    columns = "scientific_name") # cast to get a column for each taxon, and a row for each year
                       .fillna(0) # replace any NaN with 0; no larvae of that taxon were observed in that year
                       .reset_index() # make year a regular column 
            )

# show the first few rows of the new df
display(pca_data.head())

# standardize the features
features = pca_data.drop(columns=["year"]) # exclude the year column (index 0)

featuresScaled = StandardScaler().fit_transform(features) # StandardScaler's fit_transform method standardizes each column by its mean and standard deviation


Perform PCA.

In [ ]:
# apply PCA
pca = PCA(n_components=2) # how many dimensions to reduce to
principalComponents = pca.fit_transform(featuresScaled) # apply PCA on the features

principalDf = pd.DataFrame(data = principalComponents,
                           columns = ['PC1', 'PC2']) # extract PC1 and PC2

# concatenate the years to the PC1 and PC2 series
finalDf = pd.concat([principalDf, pca_data[["year"]]], axis = 1)

# show the first few years of PC1 and PC2 scores
display(finalDf.head())

# get the explained variance
print("Explained variance ratio:", pca.explained_variance_ratio_)
print("Cumulative explained variance:", np.cumsum(pca.explained_variance_ratio_))

In [ ]:
# trying to make a biplot

# define biplot function
def biplot(score, coeff, color_by, pc_names):
    # Create a scatter plot of the principal component scores
    plt.figure(figsize=(10, 8)) # set figure size
    plt.scatter(score[:, 0], score[:, 1], c = color_by, alpha=0.8) # make scatter plot
    
    # Add the feature vectors
    for i in range(coeff.shape[0]):
        plt.arrow(0, 0, coeff[i,0] * 20, coeff[i,1] * 20, color='r', alpha=0.8, head_width=0.05)
        plt.text(coeff[i,0]*23, coeff[i,1]*23, pc_names[i], color='g', ha='center', va='center')

    plt.xlabel('Principal Component 1') # set x axis label
    plt.ylabel('Principal Component 2') # set y axis label
    plt.title('PCA Biplot') # set figure title
    plt.grid(True) # add gridlines
    plt.colorbar() # add color bar
    plt.axhline(0, color='black',linewidth=0.5) # add y=0 line
    plt.axvline(0, color='black',linewidth=0.5) # add x=0 line
    plt.show() # display plot

# Get the principal component loadings
loadings = pca.components_.T  # transpose to align with features
print(loadings)

# Feature names for labeling
feature_names = features.columns
print(feature_names)

# Create the biplot
biplot(principalComponents, loadings, finalDf["year"], feature_names)

Plot PC1 timeseries.

In [ ]:
fig = plt.figure(figsize = (10, 6)) # set figure size

ax = fig.add_subplot(1, 1, 1) # subplot layout
ax.plot(finalDf["year"], finalDf["PC1"]); # variables to plot

ax.set_xlabel("Year") # set x axis label
ax.set_ylabel("PC1 Axis") # set y axis label
ax.set_title("PC1 Time Series") # set figure title

PC1 --> detrending, N* correction, first-differencing

...

***

### 4. Make Figures

Table 2: Taxa that contributed significantly to PC1 with their loadings, equivalent to their correlation with the PC1 timeseries.

Figure 2: Time series of four taxa that contributed significantly to PC1.

Figure 3: Time series of PC1 and mean midwater oxygen (200-400m).